In [1]:
using Flux, MLDatasets, Statistics
using Parameters: @with_kw
using Printf

@with_kw mutable struct Args
    learning_rate::Float64 = 3e-4       
    batch_size::Int = 1024   
    epochs::Int = 10
    verbose::Bool = true
end

Args

### "Hello World" in Julia
#### MLP on MNIST data

In [2]:
function load_data(args)
    """ Loading data """
    # Loading Dataset
    X_train, y_train = MLDatasets.MNIST.traindata(Float32)
    X_test,  y_test  = MLDatasets.MNIST.testdata(Float32)

    # Preprocessing steps
    X_train, X_test = Flux.flatten(X_train), Flux.flatten(X_test)
    y_train, y_test = Flux.onehotbatch(y_train, 0:9), Flux.onehotbatch(y_test, 0:9)

    # Batching
    train_data = Flux.Data.DataLoader(X_train, y_train, batchsize=args.batch_size, shuffle=true)
    test_data = Flux.Data.DataLoader(X_test, y_test, batchsize=args.batch_size)
    
    if args.verbose
        @printf "------Loaded Dataset statistics------\n"
        @printf "Training data amount :   %d\n" size(X_train, 2)
        @printf "Training data size   :   %d\n" size(X_train, 1)
        @printf "-------------------------------------\n"
        @printf "Testing data amount  :   %d\n" size(X_test, 2)
        @printf "Testing data size    :   %d\n" size(X_test, 1)
    end
    
    return train_data, test_data
end

function loss(model)
    """ Loss function """
    function (x, y)
        Flux.logitcrossentropy(model(x), y)
    end
end

function loss_all(data, loss_func)
    """ Calculate loss on the whole dataset """
    sum([loss_func(x, y) for (x, y) in data]) / length(data)
end

function accuracy(data, model)
    """ Calculate accuracy on the whole dataset """
    accuracy = 0
    for (x, y) in data
        predicted  = Flux.onecold(cpu(model(x)))
        true_label = Flux.onecold(cpu(y))
        accuracy += sum(Flux.onecold(cpu(model(x))) .== Flux.onecold(cpu(y))) * 1 / size(x, 2)
    end
    accuracy / length(data)
end

accuracy (generic function with 1 method)

In [6]:
args = Args()

# Loading Data
train_data, test_data = load_data(args)

# Constructing the model
model = Chain(
    Dense(28 * 28, 32, relu),
    Dense(32, 10)
)

# Training
Flux.@epochs args.epochs Flux.train!(
    loss(model), 
    params(model), 
    train_data, 
    ADAM(args.learning_rate)
)

if args.verbose
    @printf "-------Model Performance test--------\n"
    @printf "Accuracy on training data: %0.3f \n" accuracy(train_data, model) * 100
    @printf "Accuracy on testing data : %0.3f \n" accuracy(test_data, model) * 100
    @printf "-------------------------------------\n"
    @printf "Loss on training data: %0.3f \n" loss_all(train_data, loss(model)) 
    @printf "Loss on testing data : %0.3f \n" loss_all(test_data, loss(model)) 
end

------Loaded Dataset statistics------
Training data amount :   60000
Training data size   :   784
-------------------------------------
Testing data amount  :   10000
Testing data size    :   784


┌ Info: Epoch 1
└ @ Main C:\Users\alona\.julia\packages\Flux\Fj3bt\src\optimise\train.jl:121
┌ Info: Epoch 2
└ @ Main C:\Users\alona\.julia\packages\Flux\Fj3bt\src\optimise\train.jl:121
┌ Info: Epoch 3
└ @ Main C:\Users\alona\.julia\packages\Flux\Fj3bt\src\optimise\train.jl:121
┌ Info: Epoch 4
└ @ Main C:\Users\alona\.julia\packages\Flux\Fj3bt\src\optimise\train.jl:121
┌ Info: Epoch 5
└ @ Main C:\Users\alona\.julia\packages\Flux\Fj3bt\src\optimise\train.jl:121
┌ Info: Epoch 6
└ @ Main C:\Users\alona\.julia\packages\Flux\Fj3bt\src\optimise\train.jl:121
┌ Info: Epoch 7
└ @ Main C:\Users\alona\.julia\packages\Flux\Fj3bt\src\optimise\train.jl:121
┌ Info: Epoch 8
└ @ Main C:\Users\alona\.julia\packages\Flux\Fj3bt\src\optimise\train.jl:121
┌ Info: Epoch 9
└ @ Main C:\Users\alona\.julia\packages\Flux\Fj3bt\src\optimise\train.jl:121
┌ Info: Epoch 10
└ @ Main C:\Users\alona\.julia\packages\Flux\Fj3bt\src\optimise\train.jl:121


-------Model Performance test--------
Accuracy on training data: 92.248 
Accuracy on testing data : 92.366 
-------------------------------------
Loss on training data: 0.280 
Loss on testing data : 0.274 
